# Práctica tres

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica  se usará la librería de numpy, pandas, matplotlib y sklearn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn.linear_model as lm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('precio_casas.csv', sep=';')
df.head()

# Ejercicio 1

In [ ]:
data = df.to_numpy()
x = data[:, :-1] # todas las filas, todas las columnas menos la última
y = data[:, -1] # todas las filas, la última columna
fig, ax = plt.subplots(2, 4, figsize=(32, 16)) # 2 filas, 4 columnas
idx_to_r2 = [] # índice de la columna y su R2

for i, (header, vec) in enumerate(zip(df.columns, x.T)): # recorremos las columnas
    col = vec.reshape(-1, 1) # convertimos a matriz
    reg = lm.LinearRegression().fit(col, y) # ajustamos el modelo
    r2 = r2_score(y, reg.predict(col)) # calculamos el R2
    idx_to_r2.append((i, r2)) # guardamos el índice y el R2
    ax[i // 4, i % 4].scatter(vec, y, color='blue') # dibujamos los puntos
    ax[i // 4, i % 4].plot(col, col * reg.coef_[0] + reg.intercept_, color='red') # dibujamos la recta de regresión lineal
    cor = np.corrcoef(vec, y)[0, 1] # calculamos la correlación
    ax[i // 4, i % 4].set_title(f"{header}: r2={r2:.5f} cor={cor:.4f}") # título con el R2 redondeado a 5 decimales
    ax[i // 4, i % 4].set_ylim(0, 1.05 * y.max())
    ax[i // 4, i % 4].set_xlabel(header)
    ax[i // 4, i % 4].set_ylabel('Precio')

Un valor de $R^2$ cercano a 1 indica una fuerte dependencia lineal entre la variable independiente y la variable dependiente, mientras que un valor cercano a 0 indica una dependencia lineal débil o nula.
En este caso, se puede apreciar claramente como hay una correlación entre la mediana de ingresos y la del precio mayor que el resto de variables en las que la dependencia es muy débil.

# Ejercicio 2

Primero se dividen los datos en un conjunto de entrenamiento y otro de test.

In [ ]:
data = df.to_numpy()

X, y = data[:, :-1], data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print(X_train.shape, y_train.shape, )

Para obtener las columnas ordenadas segun el $R^2$.

In [ ]:
ordered_cols = [x for x, _ in sorted(idx_to_r2, key=lambda x: x[1], reverse=True)]
print(ordered_cols)

In [ ]:
rmse_collection = []

for i in range(1, 9):
    lim_cols = ordered_cols[:i]
    model = lm.LinearRegression()
    model.fit(X_train[:, lim_cols], y_train)
    y_pred = model.predict(X_test[:, lim_cols])
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    rmse_collection.append(rmse)

data = {
    'Features': df.columns[ordered_cols],
    'RMSE': rmse_collection,
}

pd.DataFrame(data)

Se puede observar que el error baja con el número de variables. Aunque se puede apreciar que sube un poco con la adición de la segunda variable comparado a si solo usase una única variable.